In [28]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [29]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# Set stock symbol and date range
STOCK_SYMBOL = 'AMZN'
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=365)

# Download historical data
df = yf.download(tickers=STOCK_SYMBOL, start=START_DATE, end=END_DATE, interval='1h')

# Save data to CSV
if not df.empty:
    df.to_csv(f'/Users/christinadarsi/Downloads/{STOCK_SYMBOL}_hourly_stock_datayf.csv')
    print(f'Data saved to {STOCK_SYMBOL}_hourly_stock_data.csv')
else:
    print("Failed to fetch data or no data available for the specified range")


[*********************100%%**********************]  1 of 1 completed

Data saved to AMZN_hourly_stock_data.csv


In [30]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# Set stock symbol and date range for Amazon (AMZN)
STOCK_SYMBOL = 'AMZN'
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=365)

# Download historical data for Amazon
df_amzn = yf.download(tickers=STOCK_SYMBOL, start=START_DATE, end=END_DATE, interval='1h')

# Save Amazon data to CSV
amzn_csv_path = f'/Users/christinadarsi/Downloads/{STOCK_SYMBOL}_hourly_stock_data.csv'
if not df_amzn.empty:
    df_amzn.to_csv(amzn_csv_path)
    print(f'Data saved to {STOCK_SYMBOL}_hourly_stock_data.csv')
else:
    print("Failed to fetch data or no data available for Amazon")

# List of additional stock symbols
additional_symbols = ['AAPL', 'MSFT', 'GOOGL', 'META', 'WMT', 'BABA']
dfs = [df_amzn[['Close']].rename(columns={'Close': 'AMZN'})]  # Include Amazon data

# Function to download and process stock data
def download_and_process_data(symbol, start_date, end_date, interval='1h'):
    df = yf.download(tickers=symbol, start=start_date, end=end_date, interval=interval)
    if not df.empty:
        df = df[['Close']].rename(columns={'Close': symbol})
        return df
    else:
        print(f"Failed to fetch data or no data available for {symbol}")
        return None

# Download data for each additional symbol
for symbol in additional_symbols:
    df = download_and_process_data(symbol, START_DATE, END_DATE)
    if df is not None:
        dfs.append(df)

# Merge all dataframes on the Datetime index
merged_df = pd.concat(dfs, axis=1)

# Save the merged dataframe to CSV
merged_csv_path = f'/Users/christinadarsi/Downloads/merged_stock_data.csv'
merged_df.to_csv(merged_csv_path)
print(f"Merged data saved to {merged_csv_path}")


[*********************100%%**********************]  1 of 1 completed


Data saved to AMZN_hourly_stock_data.csv


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Merged data saved to /Users/christinadarsi/Downloads/merged_stock_data.csv


In [31]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pandas_ta as ta
from fredapi import Fred

# Set stock symbol and date range for Amazon (AMZN)
STOCK_SYMBOL = 'AMZN'
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=365)

# FRED API key (replace 'YOUR_FRED_API_KEY' with your actual FRED API key)
FRED_API_KEY = '8e5b4a4f782c5344fb639a9108172491'

# Download historical data for Amazon
df_amzn = yf.download(tickers=STOCK_SYMBOL, start=START_DATE, end=END_DATE, interval='1h')

# Save Amazon data to CSV
amzn_csv_path = f'/Users/christinadarsi/Downloads/{STOCK_SYMBOL}_hourly_stock_data.csv'
if not df_amzn.empty:
    df_amzn.to_csv(amzn_csv_path)
    print(f'Data saved to {STOCK_SYMBOL}_hourly_stock_data.csv')
else:
    print("Failed to fetch data or no data available for Amazon")

# List of additional stock symbols
additional_symbols = ['AAPL', 'MSFT', 'GOOGL', 'META', 'WMT', 'BABA']
close_dfs = []

# Function to download and process stock data
def download_and_process_data(symbol, start_date, end_date, interval='1h'):
    df = yf.download(tickers=symbol, start=start_date, end=end_date, interval=interval)
    if not df.empty:
        df = df[['Close']].rename(columns={'Close': symbol})
        return df
    else:
        print(f"Failed to fetch data or no data available for {symbol}")
        return None

# Download data for each additional symbol
for symbol in additional_symbols:
    df = download_and_process_data(symbol, START_DATE, END_DATE)
    if df is not None:
        close_dfs.append(df)

# Combine all close value dataframes
if close_dfs:
    additional_close_df = pd.concat(close_dfs, axis=1)

    # Merge the Amazon data with additional close values
    merged_df = df_amzn.join(additional_close_df, how='left')

    # Calculate technical indicators
    merged_df['SMA_20'] = ta.sma(merged_df['Close'], length=20)
    merged_df['SMA_50'] = ta.sma(merged_df['Close'], length=50)
    merged_df['MACD'], merged_df['MACD_Signal'], merged_df['MACD_Hist'] = ta.macd(merged_df['Close'])
    merged_df['RSI'] = ta.rsi(merged_df['Close'])
    bbands = ta.bbands(merged_df['Close'], length=20)
    merged_df = merged_df.join(bbands[['BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0']])

    # Fetch economic indicators from FRED
    fred = Fred(api_key=FRED_API_KEY)

    # Interest Rates (e.g., Federal Funds Rate)
    interest_rate = fred.get_series('FEDFUNDS', start_date=START_DATE, end_date=END_DATE)
    interest_rate.index = interest_rate.index.tz_localize('UTC').tz_convert('America/New_York')
    interest_rate = interest_rate.resample('H').ffill().reindex(merged_df.index, method='ffill')
    merged_df['Interest_Rate'] = interest_rate

    # Inflation Rate (e.g., CPI)
    inflation_rate = fred.get_series('CPIAUCSL', start_date=START_DATE, end_date=END_DATE).pct_change() * 100
    inflation_rate.index = inflation_rate.index.tz_localize('UTC').tz_convert('America/New_York')
    inflation_rate = inflation_rate.resample('H').ffill().reindex(merged_df.index, method='ffill')
    merged_df['Inflation_Rate'] = inflation_rate

    # GDP Growth (e.g., Real GDP)
    gdp_growth = fred.get_series('GDPC1', start_date=START_DATE, end_date=END_DATE).pct_change() * 100
    gdp_growth.index = gdp_growth.index.tz_localize('UTC').tz_convert('America/New_York')
    gdp_growth = gdp_growth.resample('H').ffill().reindex(merged_df.index, method='ffill')
    merged_df['GDP_Growth'] = gdp_growth

    # Unemployment Rate
    unemployment_rate = fred.get_series('UNRATE', start_date=START_DATE, end_date=END_DATE)
    unemployment_rate.index = unemployment_rate.index.tz_localize('UTC').tz_convert('America/New_York')
    unemployment_rate = unemployment_rate.resample('H').ffill().reindex(merged_df.index, method='ffill')
    merged_df['Unemployment_Rate'] = unemployment_rate

    # Save the merged dataframe with technical indicators and economic indicators to CSV
    merged_csv_path = f'/Users/christinadarsi/Downloads/merged_stock_data_with_indicators_and_economics.csv'
    merged_df.to_csv(merged_csv_path)
    print(f"Merged data with technical and economic indicators saved to {merged_csv_path}")
else:
    print("No additional data was fetched to merge with Amazon data.")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Data saved to AMZN_hourly_stock_data.csv


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Merged data with technical and economic indicators saved to /Users/christinadarsi/Downloads/merged_stock_data_with_indicators_and_economics.csv


In [32]:
!pip install yahoo_fin pandas_ta
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pandas_ta as ta
from fredapi import Fred

# Set stock symbol and date range for Amazon (AMZN)
STOCK_SYMBOL = 'AMZN'
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=365)

# FRED API key (replace 'YOUR_FRED_API_KEY' with your actual FRED API key)
FRED_API_KEY = '8e5b4a4f782c5344fb639a9108172491'

# Download historical data for Amazon
df_amzn = yf.download(tickers=STOCK_SYMBOL, start=START_DATE, end=END_DATE, interval='1h')

# Save Amazon data to CSV
amzn_csv_path = f'/Users/christinadarsi/Downloads/{STOCK_SYMBOL}_hourly_stock_data.csv'
if not df_amzn.empty:
    df_amzn.to_csv(amzn_csv_path)
    print(f'Data saved to {STOCK_SYMBOL}_hourly_stock_data.csv')
else:
    print("Failed to fetch data or no data available for Amazon")

# List of additional stock symbols
additional_symbols = ['AAPL', 'MSFT', 'GOOGL', 'META', 'WMT', 'BABA']
close_dfs = []

# Function to download and process stock data
def download_and_process_data(symbol, start_date, end_date, interval='1h'):
    df = yf.download(tickers=symbol, start=start_date, end=end_date, interval=interval)
    if not df.empty:
        df = df[['Close']].rename(columns={'Close': symbol})
        return df
    else:
        print(f"Failed to fetch data or no data available for {symbol}")
        return None

# Download data for each additional symbol
for symbol in additional_symbols:
    df = download_and_process_data(symbol, START_DATE, END_DATE)
    if df is not None:
        close_dfs.append(df)

# Combine all close value dataframes
if close_dfs:
    additional_close_df = pd.concat(close_dfs, axis=1)

    # Merge the Amazon data with additional close values
    merged_df = df_amzn.join(additional_close_df, how='left')

    # Calculate technical indicators
    merged_df['SMA_20'] = ta.sma(merged_df['Close'], length=20)
    merged_df['SMA_50'] = ta.sma(merged_df['Close'], length=50)
    merged_df['MACD'], merged_df['MACD_Signal'], merged_df['MACD_Hist'] = ta.macd(merged_df['Close'])
    merged_df['RSI'] = ta.rsi(merged_df['Close'])
    bbands = ta.bbands(merged_df['Close'], length=20)
    merged_df = merged_df.join(bbands[['BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0']])

    # Fetch economic indicators from FRED
    fred = Fred(api_key=FRED_API_KEY)

    # Interest Rates (e.g., Federal Funds Rate)
    interest_rate = fred.get_series('FEDFUNDS', start_date=START_DATE, end_date=END_DATE)
    interest_rate.index = interest_rate.index.tz_localize('UTC').tz_convert('America/New_York')
    interest_rate = interest_rate.resample('H').ffill().reindex(merged_df.index, method='ffill')
    merged_df['Interest_Rate'] = interest_rate

    # Inflation Rate (e.g., CPI)
    inflation_rate = fred.get_series('CPIAUCSL', start_date=START_DATE, end_date=END_DATE).pct_change() * 100
    inflation_rate.index = inflation_rate.index.tz_localize('UTC').tz_convert('America/New_York')
    inflation_rate = inflation_rate.resample('H').ffill().reindex(merged_df.index, method='ffill')
    merged_df['Inflation_Rate'] = inflation_rate

    # GDP Growth (e.g., Real GDP)
    gdp_growth = fred.get_series('GDPC1', start_date=START_DATE, end_date=END_DATE).pct_change() * 100
    gdp_growth.index = gdp_growth.index.tz_localize('UTC').tz_convert('America/New_York')
    gdp_growth = gdp_growth.resample('H').ffill().reindex(merged_df.index, method='ffill')
    merged_df['GDP_Growth'] = gdp_growth

    # Unemployment Rate
    unemployment_rate = fred.get_series('UNRATE', start_date=START_DATE, end_date=END_DATE)
    unemployment_rate.index = unemployment_rate.index.tz_localize('UTC').tz_convert('America/New_York')
    unemployment_rate = unemployment_rate.resample('H').ffill().reindex(merged_df.index, method='ffill')
    merged_df['Unemployment_Rate'] = unemployment_rate

    # Fetch company financials using yfinance
    def fetch_company_financials(symbol):
        ticker = yf.Ticker(symbol)
        financials = ticker.financials
        balance_sheet = ticker.balance_sheet
        cashflow = ticker.cashflow
        return financials, balance_sheet, cashflow

    financials, balance_sheet, cashflow = fetch_company_financials(STOCK_SYMBOL)

    # Extract relevant financial data
    revenue = financials.loc['Total Revenue'].values[0]
    net_income = financials.loc['Net Income'].values[0]
    gross_profit = financials.loc['Gross Profit'].values[0]
    total_debt = balance_sheet.loc['Total Liabilities Net Minority Interest'].values[0]
    operating_cash_flow = cashflow.loc['Operating Cash Flow'].values[0]

    # Add financial data to the merged dataframe
    merged_df['Revenue'] = revenue
    merged_df['Net_Income'] = net_income
    merged_df['Gross_Profit'] = gross_profit
    merged_df['Total_Debt'] = total_debt
    merged_df['Operating_Cash_Flow'] = operating_cash_flow

    # Save the merged dataframe with technical indicators, economic indicators, and financial data to CSV
    merged_csv_path = f'/Users/christinadarsi/Downloads/merged_stock_data_with_indicators_economics_financials.csv'
    merged_df.to_csv(merged_csv_path)
    print(f"Merged data with technical, economic, and financial indicators saved to {merged_csv_path}")
else:
    print("No additional data was fetched to merge with Amazon data.")









[*********************100%%**********************]  1 of 1 completed


Data saved to AMZN_hourly_stock_data.csv


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Merged data with technical, economic, and financial indicators saved to /Users/christinadarsi/Downloads/merged_stock_data_with_indicators_economics_financials.csv
